<a href="https://colab.research.google.com/github/Rathore25/SapiensAI/blob/main/%20Training_SapiensQA.ipynb%20" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#!pip install cdqa

In [1]:
import os
import torch
import joblib
#from cdqa.reader import BertProcessor, BertQA
#from cdqa.utils.download import download_squad, download_model, download_bnpp_data
from cdqa.pipeline import QAPipeline
import json
import pandas as pd
import requests
import wget

/usr/local/lib/python3.7/dist-packages/tqdm/autonotebook/__init__.py:18: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  " (e.g. in jupyter console)", TqdmExperimentalWarning)


In [2]:
# directory = './data'

# # Downloading data
# download_squad(dir=directory)
# download_bnpp_data(dir=directory)
# # Downloading pre-trained BERT fine-tuned on SQuAD 1.1
# download_model('bert-squad_1.1', dir=directory)
# # Downloading pre-trained DistilBERT fine-tuned on SQuAD 1.1
# download_model('distilbert-squad_1.1', dir=directory)

In [3]:
response = requests.get('https://docs.google.com/uc?export=download&id=1oSUFKZMao_gQxGDpCZuRXV6EULjFjmoZ')
sapiens_original = response.json()

response = requests.get('https://docs.google.com/uc?export=download&id=1b5xy1Z4EuFVXkMOQIupl27a1kRNPguVr')
sapiens_annotated = response.json()
with open('./sapiens_annotated.json', 'w') as file:
  json.dump(sapiens_annotated, file)

In [4]:
dictionary_df = []

for item in sapiens_original['data']:
  title = item['title']
  paragraphs = []

  for paragraph in item['paragraphs']:
    paragraphs.append(paragraph['context'])
  
  dictionary_df.append({'title':title, 'paragraphs':paragraphs})

In [5]:
df = pd.DataFrame(dictionary_df)

In [6]:
print(df)

                                                title                                         paragraphs
0                           Timeline of History Years  [13.5 billion years from present, matter and e...
1                        An Animal of No Significance  [ABOUT 13.5 BILLION YEARS AGO, MATTER, energy,...
2   An Animal of No Significance - The Cost of Thi...  [Despite their many differences, all human spe...
3      An Animal of No Significance - A Race of Cooks  [A significant step on the way to the top was ...
4                               The Tree of Knowledge  [Despite the benefits of fire, 150,000 years a...
..                                                ...                                                ...
85          The End of Homo Sapiens - Of Mice and Men  [Biological engineering is deliberate human in...
86  The End of Homo Sapiens - The Return of the Ne...  [But geneticists do not only want to transform...
87              The End of Homo Sapiens - Bionic Life  

In [16]:
#wget.download(url='https://github.com/cdqa-suite/cdQA/releases/download/bert_qa/bert_qa.joblib', out='./')
wget.download(url='https://github.com/cdqa-suite/cdQA/releases/download/distilbert_qa/distilbert_qa.joblib', out='./')

'.//distilbert_qa.joblib'

In [17]:
cdqa_pipeline = QAPipeline(reader='./distilbert_qa.joblib') # use 'distilbert_qa.joblib' for DistilBERT instead of BERT
cdqa_pipeline.fit_retriever(df=df)

QAPipeline(reader=BertQA(adam_epsilon=1e-08,
                         bert_model='distilbert-base-uncased',
                         do_lower_case=True, fp16=False,
                         gradient_accumulation_steps=1, learning_rate=5e-05,
                         local_rank=-1, loss_scale=0, max_answer_length=30,
                         n_best_size=20, no_cuda=False,
                         null_score_diff_threshold=0.0, num_train_epochs=3.0,
                         output_dir=None, predict_batch_size=8, seed=42,
                         server_ip='', ser...size=8,
                         verbose_logging=False, version_2_with_negative=False,
                         warmup_proportion=0.1, warmup_steps=0),
           retrieve_by_doc=False,
           retriever=BM25Retriever(b=0.75, floor=None, k1=2.0, lowercase=True,
                                   max_df=0.85, min_df=2, ngram_range=(1, 2),
                                   preprocessor=None, stop_words='english',
           

In [18]:
cdqa_pipeline.fit_reader('./sapiens_annotated.json')

Could not find answer: '' vs. '70,000 years from present'
Could not find answer: '' vs. '200 years ago'
Could not find answer: '' vs. 'Once the fire died down, Stone Age entrepreneurs could walk through the smoking remains and harvest charcoaled animals, nuts and tubers.'
Could not find answer: '' vs. 'The opposing view, called the ‘Replacement Theory’ tells a very different story – one of incompatibility, revulsion, and perhaps even genocide. According to this theory, Sapiens and other humans had different anatomies, and most likely different mating habits and even body odours. They would have had little sexual interest in one another. According to this view, Sapiens replaced all the previous human populations without merging with them. If that is the case, the lineages of all contemporary humans can be traced back, exclusively, to East Africa, 70,000 years ago. We are all ‘pure Sapiens’.'
Could not find answer: '' vs. 'There is some evidence that the size of the average Sapiens brain

QAPipeline(reader=BertQA(adam_epsilon=1e-08,
                         bert_model='distilbert-base-uncased',
                         do_lower_case=True, fp16=False,
                         gradient_accumulation_steps=1, learning_rate=5e-05,
                         local_rank=-1, loss_scale=0, max_answer_length=30,
                         n_best_size=20, no_cuda=False,
                         null_score_diff_threshold=0.0, num_train_epochs=3.0,
                         output_dir=None, predict_batch_size=8, seed=42,
                         server_ip='', ser...size=8,
                         verbose_logging=False, version_2_with_negative=False,
                         warmup_proportion=0.1, warmup_steps=0),
           retrieve_by_doc=False,
           retriever=BM25Retriever(b=0.75, floor=None, k1=2.0, lowercase=True,
                                   max_df=0.85, min_df=2, ngram_range=(1, 2),
                                   preprocessor=None, stop_words='english',
           

In [19]:
cdqa_pipeline.dump_reader('./sapiens_distilbert.joblib')

In [20]:
# Answered by Bert and Distilbert
cdqa_pipeline.predict(query='Why do empires not work?')

('In the long run, it is not possible to rule effectively over a large number of conquered peoples.',
 'Imperial Visions - Evil Empires?',
 '1.\tEmpires do not work. In the long run, it is not possible to rule effectively over a large number of conquered peoples.',
 9.932559288062196)

In [12]:
# Answered by Bert
cdqa_pipeline.predict(query="When did sapiens come to Australia?")

('45,000 years from present, Sapiens settle Australia',
 'Timeline of History Years',
 '45,000 years from present, Sapiens settle Australia. Extinction of Australian megafauna.',
 11.197714515345469)

In [21]:
# Answered by Distilbert
cdqa_pipeline.predict(query="When did sapiens come to Australia?")

('45,000 years from present',
 'Timeline of History Years',
 '45,000 years from present, Sapiens settle Australia. Extinction of Australian megafauna.',
 10.580057807581795)

In [14]:
# Answered by Bert
cdqa_pipeline.predict(query='What can you tell me about Sapiens?')

('It’s a secret that only our tribe and the moon spirits are allowed to know.',
 'The Marriage of Science and Empire - The Mentality of Conquest',
 '‘Oh, I cannot tell you. It’s a secret that only our tribe and the moon spirits are allowed to know.’',
 8.112579348755647)

In [22]:
# Answered by Distilbert
cdqa_pipeline.predict(query='What can you tell me about Sapiens?')

('Oh, I cannot tell you',
 'The Marriage of Science and Empire - The Mentality of Conquest',
 '‘Oh, I cannot tell you. It’s a secret that only our tribe and the moon spirits are allowed to know.’',
 6.579646399689484)